# Class Example: Understanding Joins in Polars

This notebook demonstrates how to perform joins in Polars with practical examples.

In [1]:
# Import necessary libraries
import polars as pl
from datetime import date

# Set display options for better readability
pl.Config.set_tbl_rows(20)
pl.Config.set_fmt_str_lengths(50)

print("Polars version:", pl.__version__)
print("Setup complete! Let's explore joins.")

Polars version: 1.25.2
Setup complete! Let's explore joins.


## 1. Creating Sample Datasets

Let's create realistic datasets to demonstrate joins. We'll simulate a retail business with customers, products, and orders.

In [3]:
# Create customers dataset
customers = pl.DataFrame({
    "customer_id": [1, 2, 3, 4, 5, 6],
    "name": ["Alice Smith", "Bob Johnson", "Charlie Brown", "Diana Ross", "Eve Davis", "Frank Miller"],
    "email": ["alice@email.com", "bob@email.com", "charlie@email.com",
              "diana@email.com", "eve@email.com", "frank@email.com"],
    "join_date": [date(2023, 1, 15), date(2023, 2, 20), date(2023, 3, 10),
                  date(2023, 4, 5), date(2023, 5, 12), date(2023, 6, 30)],
    "customer_type": ["Premium", "Standard", "Premium", "Standard", "Premium", "Standard"]
})


print(f"\nPrimary Key: customer_id (unique identifier for each customer)")
print(f"Total customers: {len(customers)}")
print("Customers Table:")
customers


Primary Key: customer_id (unique identifier for each customer)
Total customers: 6
Customers Table:


customer_id,name,email,join_date,customer_type
i64,str,str,date,str
1,"""Alice Smith""","""alice@email.com""",2023-01-15,"""Premium"""
2,"""Bob Johnson""","""bob@email.com""",2023-02-20,"""Standard"""
3,"""Charlie Brown""","""charlie@email.com""",2023-03-10,"""Premium"""
4,"""Diana Ross""","""diana@email.com""",2023-04-05,"""Standard"""
5,"""Eve Davis""","""eve@email.com""",2023-05-12,"""Premium"""
6,"""Frank Miller""","""frank@email.com""",2023-06-30,"""Standard"""


In [5]:
# Create products dataset
products = pl.DataFrame({
    "product_id": ["P001", "P002", "P003", "P004", "P005"],
    "product_name": ["Laptop", "Mouse", "Keyboard", "Monitor", "Headphones"],
    "category": ["Electronics", "Accessories", "Accessories", "Electronics", "Accessories"],
    "price": [1200.00, 25.00, 80.00, 300.00, 150.00],
    "stock_quantity": [50, 200, 150, 75, 100]
})


print(f"\nPrimary Key: product_id (unique identifier for each product)")
print(f"Total products: {len(products)}")
print("Products Table:")
products


Primary Key: product_id (unique identifier for each product)
Total products: 5
Products Table:


product_id,product_name,category,price,stock_quantity
str,str,str,f64,i64
"""P001""","""Laptop""","""Electronics""",1200.0,50
"""P002""","""Mouse""","""Accessories""",25.0,200
"""P003""","""Keyboard""","""Accessories""",80.0,150
"""P004""","""Monitor""","""Electronics""",300.0,75
"""P005""","""Headphones""","""Accessories""",150.0,100


In [6]:
# Create orders dataset (notice the foreign keys)
orders = pl.DataFrame({
    "order_id": [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    "customer_id": [1, 2, 1, 3, 4, 2, 5, 1, 3, 7],  # Note: customer_id 7 doesn't exist!
    "product_id": ["P001", "P002", "P003", "P001", "P004", "P005", "P002", "P004", "P006", "P001"],  # Note: P006 doesn't exist!
    "order_date": [date(2023, 7, 1), date(2023, 7, 2), date(2023, 7, 3), date(2023, 7, 4),
                   date(2023, 7, 5), date(2023, 7, 6), date(2023, 7, 7), date(2023, 7, 8),
                   date(2023, 7, 9), date(2023, 7, 10)],
    "quantity": [1, 2, 1, 1, 2, 1, 3, 1, 1, 1],
    "total_amount": [1200.00, 50.00, 80.00, 1200.00, 600.00, 150.00, 75.00, 300.00, 0.00, 1200.00]
})


print(f"\nForeign Keys: customer_id (references customers), product_id (references products)")
print(f"Total orders: {len(orders)}")
print("\nNote: This dataset intentionally includes problematic data for demonstration!")
print("Orders Table:")
orders


Foreign Keys: customer_id (references customers), product_id (references products)
Total orders: 10

Note: This dataset intentionally includes problematic data for demonstration!
Orders Table:


order_id,customer_id,product_id,order_date,quantity,total_amount
i64,i64,str,date,i64,f64
101,1,"""P001""",2023-07-01,1,1200.0
102,2,"""P002""",2023-07-02,2,50.0
103,1,"""P003""",2023-07-03,1,80.0
104,3,"""P001""",2023-07-04,1,1200.0
105,4,"""P004""",2023-07-05,2,600.0
106,2,"""P005""",2023-07-06,1,150.0
107,5,"""P002""",2023-07-07,3,75.0
108,1,"""P004""",2023-07-08,1,300.0
109,3,"""P006""",2023-07-09,1,0.0


## 2. Understanding Join Keys

Before joining, always examine your join keys!

In [7]:
# Check uniqueness of primary keys
print("Checking Primary Keys:")
print(f"Unique customer_ids: {customers['customer_id'].n_unique()} (should equal {len(customers)})")
print(f"Unique product_ids: {products['product_id'].n_unique()} (should equal {len(products)})")
print(f"Unique order_ids: {orders['order_id'].n_unique()} (should equal {len(orders)})")

print("\nChecking Foreign Keys in Orders:")
print(f"Unique customer_ids in orders: {orders['customer_id'].n_unique()}")
print(f"Unique product_ids in orders: {orders['product_id'].n_unique()}")

Checking Primary Keys:
Unique customer_ids: 6 (should equal 6)
Unique product_ids: 5 (should equal 5)
Unique order_ids: 10 (should equal 10)

Checking Foreign Keys in Orders:
Unique customer_ids in orders: 6
Unique product_ids in orders: 6


In [8]:
# Identify problematic data
print("Checking for mismatched keys:")

# Customer IDs in orders but not in customers
orders_customer_ids = set(orders["customer_id"].unique().to_list())
customers_ids = set(customers["customer_id"].unique().to_list())
orphan_customers = orders_customer_ids - customers_ids
print(f"\nCustomer IDs in orders but not in customers table: {orphan_customers}")

# Product IDs in orders but not in products
orders_product_ids = set(orders["product_id"].unique().to_list())
products_ids = set(products["product_id"].unique().to_list())
orphan_products = orders_product_ids - products_ids
print(f"Product IDs in orders but not in products table: {orphan_products}")

# Customers with no orders
customers_no_orders = customers_ids - orders_customer_ids
print(f"\nCustomer IDs with no orders: {customers_no_orders}")

Checking for mismatched keys:

Customer IDs in orders but not in customers table: {7}
Product IDs in orders but not in products table: {'P006'}

Customer IDs with no orders: {6}


## 3. Inner Join Example

Inner join returns only matching records from both tables.

In [10]:
# Inner join: Orders with Customer information
orders_with_customers_inner = orders.join(
    customers,
    on="customer_id",
    how="inner"
)


print(f"\nRows in result: {len(orders_with_customers_inner)}")
print(f"Original orders: {len(orders)}")
print(f"Lost rows: {len(orders) - len(orders_with_customers_inner)} (order with customer_id=7)")
print("Inner Join: Orders + Customers")
orders_with_customers_inner


Rows in result: 9
Original orders: 10
Lost rows: 1 (order with customer_id=7)
Inner Join: Orders + Customers


order_id,customer_id,product_id,order_date,quantity,total_amount,name,email,join_date,customer_type
i64,i64,str,date,i64,f64,str,str,date,str
101,1,"""P001""",2023-07-01,1,1200.0,"""Alice Smith""","""alice@email.com""",2023-01-15,"""Premium"""
102,2,"""P002""",2023-07-02,2,50.0,"""Bob Johnson""","""bob@email.com""",2023-02-20,"""Standard"""
103,1,"""P003""",2023-07-03,1,80.0,"""Alice Smith""","""alice@email.com""",2023-01-15,"""Premium"""
104,3,"""P001""",2023-07-04,1,1200.0,"""Charlie Brown""","""charlie@email.com""",2023-03-10,"""Premium"""
105,4,"""P004""",2023-07-05,2,600.0,"""Diana Ross""","""diana@email.com""",2023-04-05,"""Standard"""
106,2,"""P005""",2023-07-06,1,150.0,"""Bob Johnson""","""bob@email.com""",2023-02-20,"""Standard"""
107,5,"""P002""",2023-07-07,3,75.0,"""Eve Davis""","""eve@email.com""",2023-05-12,"""Premium"""
108,1,"""P004""",2023-07-08,1,300.0,"""Alice Smith""","""alice@email.com""",2023-01-15,"""Premium"""
109,3,"""P006""",2023-07-09,1,0.0,"""Charlie Brown""","""charlie@email.com""",2023-03-10,"""Premium"""


In [ ]:
# Which orders were excluded?
excluded_orders = orders.filter(
    ~pl.col("order_id").is_in(orders_with_customers_inner["order_id"])
)
print("Orders excluded by inner join:")
print(excluded_orders)

## 4. Left Join Example

Left join keeps all records from the left table, adding nulls where no match exists.

In [11]:
# Left join: Keep all orders, add customer info where available
orders_with_customers_left = orders.join(
    customers,
    on="customer_id",
    how="left"
)


print(f"\nRows in result: {len(orders_with_customers_left)} (same as original orders)")
print("\nNotice the NULL values for customer_id=7!")
print("Left Join: Orders + Customers")
orders_with_customers_left


Rows in result: 10 (same as original orders)

Notice the NULL values for customer_id=7!
Left Join: Orders + Customers


order_id,customer_id,product_id,order_date,quantity,total_amount,name,email,join_date,customer_type
i64,i64,str,date,i64,f64,str,str,date,str
101,1,"""P001""",2023-07-01,1,1200.0,"""Alice Smith""","""alice@email.com""",2023-01-15,"""Premium"""
102,2,"""P002""",2023-07-02,2,50.0,"""Bob Johnson""","""bob@email.com""",2023-02-20,"""Standard"""
103,1,"""P003""",2023-07-03,1,80.0,"""Alice Smith""","""alice@email.com""",2023-01-15,"""Premium"""
104,3,"""P001""",2023-07-04,1,1200.0,"""Charlie Brown""","""charlie@email.com""",2023-03-10,"""Premium"""
105,4,"""P004""",2023-07-05,2,600.0,"""Diana Ross""","""diana@email.com""",2023-04-05,"""Standard"""
106,2,"""P005""",2023-07-06,1,150.0,"""Bob Johnson""","""bob@email.com""",2023-02-20,"""Standard"""
107,5,"""P002""",2023-07-07,3,75.0,"""Eve Davis""","""eve@email.com""",2023-05-12,"""Premium"""
108,1,"""P004""",2023-07-08,1,300.0,"""Alice Smith""","""alice@email.com""",2023-01-15,"""Premium"""
109,3,"""P006""",2023-07-09,1,0.0,"""Charlie Brown""","""charlie@email.com""",2023-03-10,"""Premium"""


In [ ]:
# Find orders with no matching customer
orders_no_customer = orders_with_customers_left.filter(
    pl.col("name").is_null()
)
print("Orders with no matching customer (after left join):")
orders_no_customer

In [ ]:
# Create a customer preferences table with duplicate customer_ids
customer_preferences = pl.DataFrame({
    "customer_id": [1, 1, 2, 3, 3, 3],  # Duplicates!
    "preference_type": ["email", "sms", "email", "email", "sms", "phone"],
    "preference_value": ["weekly", "never", "daily", "weekly", "daily", "weekly"]
})

print("Customer Preferences (with duplicates):")
print(customer_preferences)
print(f"\nNotice customer_id appears multiple times!")

In [18]:
# customers table w/ duplicate "Primary" keys
customers_dups = pl.DataFrame({
    "customer_id": [1, 1, 2, 3, 4, 5, 6],
    "name": ["Alice Smith", "Alice Smith", "Bob Johnson", "Charlie Brown", "Diana Ross", "Eve Davis", "Frank Miller"],
    "email": ["alice@email.com", "alice@email.com", "bob@email.com", "charlie@email.com",
              "diana@email.com", "eve@email.com", "frank@email.com"],
    "join_date": [date(2023, 1, 15), date(2023, 1, 15), date(2023, 2, 20), date(2023, 3, 10),
                  date(2023, 4, 5), date(2023, 5, 12), date(2023, 6, 30)],
    "customer_type": ["Premium", "Premium", "Standard", "Premium", "Standard", "Premium", "Standard"]
})
customers_dups


customer_id,name,email,join_date,customer_type
i64,str,str,date,str
1,"""Alice Smith""","""alice@email.com""",2023-01-15,"""Premium"""
1,"""Alice Smith""","""alice@email.com""",2023-01-15,"""Premium"""
2,"""Bob Johnson""","""bob@email.com""",2023-02-20,"""Standard"""
3,"""Charlie Brown""","""charlie@email.com""",2023-03-10,"""Premium"""
4,"""Diana Ross""","""diana@email.com""",2023-04-05,"""Standard"""
5,"""Eve Davis""","""eve@email.com""",2023-05-12,"""Premium"""
6,"""Frank Miller""","""frank@email.com""",2023-06-30,"""Standard"""


In [21]:
# Inner join customers_dups with orders
customers_dups_with_orders = customers_dups.join(
    orders,
    on="customer_id",
    how="inner"
)

# Notice the row explosion here!
print(f"\nRows in result: {len(customers_dups_with_orders)}")
print(f"Original customers_dups: {len(customers_dups)}")
print(f"Original orders: {len(orders)}")
print("Customers (with duplicates) joined with Orders (Inner Join):")
customers_dups_with_orders


Rows in result: 13
Original customers_dups: 7
Original orders: 10
Customers (with duplicates) joined with Orders (Inner Join):


customer_id,name,email,join_date,customer_type,order_id,product_id,order_date,quantity,total_amount
i64,str,str,date,str,i64,str,date,i64,f64
1,"""Alice Smith""","""alice@email.com""",2023-01-15,"""Premium""",101,"""P001""",2023-07-01,1,1200.0
1,"""Alice Smith""","""alice@email.com""",2023-01-15,"""Premium""",103,"""P003""",2023-07-03,1,80.0
1,"""Alice Smith""","""alice@email.com""",2023-01-15,"""Premium""",108,"""P004""",2023-07-08,1,300.0
1,"""Alice Smith""","""alice@email.com""",2023-01-15,"""Premium""",101,"""P001""",2023-07-01,1,1200.0
1,"""Alice Smith""","""alice@email.com""",2023-01-15,"""Premium""",103,"""P003""",2023-07-03,1,80.0
1,"""Alice Smith""","""alice@email.com""",2023-01-15,"""Premium""",108,"""P004""",2023-07-08,1,300.0
2,"""Bob Johnson""","""bob@email.com""",2023-02-20,"""Standard""",102,"""P002""",2023-07-02,2,50.0
2,"""Bob Johnson""","""bob@email.com""",2023-02-20,"""Standard""",106,"""P005""",2023-07-06,1,150.0
3,"""Charlie Brown""","""charlie@email.com""",2023-03-10,"""Premium""",104,"""P001""",2023-07-04,1,1200.0


In [20]:
#compare to original orders table
orders

order_id,customer_id,product_id,order_date,quantity,total_amount
i64,i64,str,date,i64,f64
101,1,"""P001""",2023-07-01,1,1200.0
102,2,"""P002""",2023-07-02,2,50.0
103,1,"""P003""",2023-07-03,1,80.0
104,3,"""P001""",2023-07-04,1,1200.0
105,4,"""P004""",2023-07-05,2,600.0
106,2,"""P005""",2023-07-06,1,150.0
107,5,"""P002""",2023-07-07,3,75.0
108,1,"""P004""",2023-07-08,1,300.0
109,3,"""P006""",2023-07-09,1,0.0


## 8. Multiple Joins

Often you need to join multiple tables to get complete information.

In [ ]:
# Complete order information: Orders + Customers + Products
complete_orders = (
    orders
    .join(customers, on="customer_id", how="left")
    .join(products, on="product_id", how="left")
    .select([
        pl.col("order_id"),
        pl.col("order_date"),
        pl.col("customer_id"),
        pl.col("name").alias("customer_name"),
        pl.col("customer_type"),
        pl.col("product_id"),
        pl.col("product_name"),
        pl.col("category"),
        pl.col("quantity"),
        pl.col("price"),
        pl.col("total_amount")
    ])
)

print("Complete Order Information (Multiple Joins):")
print(complete_orders)
print(f"\nNotice the NULL values where customer_id=7 or product_id='P006'")

## 9. Validating Joins

Always validate your joins to ensure correctness!

In [ ]:
# Validation checks
print("Join Validation:")
print(f"Original orders: {len(orders)}")
print(f"After joins: {len(complete_orders)}")
print(f"Rows match: {len(orders) == len(complete_orders)}")

# Check for data quality issues
print("\nData Quality Issues:")
missing_customers = complete_orders.filter(pl.col("customer_name").is_null()).select("order_id", "customer_id")
print(f"Orders with missing customer info: {len(missing_customers)}")
if len(missing_customers) > 0:
    print(missing_customers)

missing_products = complete_orders.filter(pl.col("product_name").is_null()).select("order_id", "product_id")
print(f"\nOrders with missing product info: {len(missing_products)}")
if len(missing_products) > 0:
    print(missing_products)

## 10. Business Analysis with Joins

Now let's use joins to answer business questions.

In [ ]:
# Question 1: What is the total revenue by customer type?
revenue_by_type = (
    orders
    .join(customers, on="customer_id", how="inner")
    .group_by("customer_type")
    .agg([
        pl.col("total_amount").sum().alias("total_revenue"),
        pl.len().alias("order_count"),
        pl.col("customer_id").n_unique().alias("unique_customers")
    ])
    .sort("total_revenue", descending=True)
)

print("Revenue by Customer Type:")
print(revenue_by_type)

In [ ]:
# Question 2: Which products are ordered most by Premium customers?
premium_product_analysis = (
    orders
    .join(customers.filter(pl.col("customer_type") == "Premium"), on="customer_id", how="inner")
    .join(products, on="product_id", how="inner")
    .group_by(["product_name", "category"])
    .agg([
        pl.col("quantity").sum().alias("total_quantity"),
        pl.len().alias("order_count"),
        pl.col("total_amount").sum().alias("revenue")
    ])
    .sort("revenue", descending=True)
)

print("Products Popular with Premium Customers:")
print(premium_product_analysis)

In [ ]:
# Question 3: Customer purchase summary (including those with no purchases)
customer_summary = (
    customers
    .join(orders, on="customer_id", how="left")
    .group_by(["customer_id", "name", "customer_type", "join_date"])
    .agg([
        pl.col("order_id").count().alias("total_orders"),
        pl.col("total_amount").sum().alias("total_spent"),
        pl.col("order_date").min().alias("first_order"),
        pl.col("order_date").max().alias("last_order")
    ])
    .with_columns([
        # Fill nulls for customers with no orders
        pl.col("total_spent").fill_null(0),
        pl.col("total_orders").fill_null(0)
    ])
    .sort("total_spent", descending=True)
)

print("Customer Purchase Summary (including non-purchasers):")
print(customer_summary)

## Summary

Key takeaways from this lesson:

1. **Always validate your join keys** before joining
2. **Choose the correct join type** based on your business needs
3. **Watch out for duplicate keys** that can cause row explosion
4. **Check row counts** after joins to ensure correctness
5. **Handle NULL values** appropriately in your analysis

Practice these concepts in the lab exercise!